# Test preproc file construction to avoid FITS errors


## Replicate `desi_preproc` script

Example call to `desi_preproc`:
```
desi_preproc -n 20210412 -e 84523 --cameras r0 \
    --fibermap $DESI_ROOT/spectro/redux/f3/preproc/20210412/00084523/fibermap-00084523.fits \
    --outfile preproc-r0-00084523.fits --model-variance
```
For details see [desihub/desidatamodel#87](https://github.com/desihub/desidatamodel/issues/87).

## Running `desi_preproc` in `f3`.

* Log file: `/global/cfs/cdirs/desi/spectro/redux/f3/run/scripts/night/20210412/prestdstar-20210412-00084523-a0123456789-51794302.log`
* Batch file: `/global/cfs/cdirs/desi/spectro/redux/f3/run/scripts/night/20210412/prestdstar-20210412-00084523-a0123456789.slurm`
* srun command: `srun -N 3 -n 200 -c 4 --cpu-bind=cores desi_proc   --traceshift --nostdstarfit --nofluxcalib --cameras a0123456789 -n 20210412 -e 84523 --timingfile /global/cfs/cdirs/desi/spectro/redux/f3/run/scripts/night/20210412/prestdstar-20210412-00084523-a0123456789-timing-$SLURM_JOBID.json --mpi --nofluxcalib`

## Set up options

In [2]:
import os
import subprocess as sub
import time
import warnings
import fitsio
import numpy as np
from astropy.io import fits
from astropy.table import Table
from desiutil.depend import add_dependencies
from desiutil.log import get_logger
from desispec.calibfinder import parse_date_obs, CalibFinder
from desispec.io import findfile, read_raw, write_image, iotime, read_fibermap, read_xytraceset
from desispec.io.util import addkeys, fitsheader
from desispec.util import header2night
from desispec.preproc import preproc, parse_sec_keyword
import desispec.maskbits as maskbits


In [3]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
night = 20210412
expid = 84523
camera = 'r0'
infile = findfile('raw', night, expid)
fibermap = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'], 'preproc', str(night), f'{expid:08d}', f'fibermap-{expid:08d}.fits')
outfile = os.path.join(os.environ['CSCRATCH'], f'preproc-{camera}-{expid:08d}.fits')
infile, fibermap, outfile

DEBUG:meta.py:172:findfile: hpixdir = 'hpix'
DEBUG:meta.py:183:findfile: rawdata_dir = '/global/cfs/cdirs/desi/spectro/data'


('/global/cfs/cdirs/desi/spectro/data/20210412/00084523/desi-00084523.fits.fz',
 '/global/cfs/cdirs/desi/spectro/redux/f3/preproc/20210412/00084523/fibermap-00084523.fits',
 '/global/cscratch1/sd/bweaver/preproc-r0-00084523.fits')

## Read raw data

`desi_preproc` executable corresponds to `desispec.scripts.preproc.main()`, which itself primarily coordinates multiprocessing based on the single-file function `desispec.scripts.preproc.preproc_file()`.  `preproc_file()` itself does very little except call `desispec.io.raw.read_raw()` and `desispec.io.image.write_image()`.

In [ ]:
#
# Interesting design choice: even though fill_header=None, it still fills from HDU 0.  It is triggered by the fact that the keyword exists, not by its value.  Very IDL.
#
img = read_raw(infile, camera, fibermapfile=fibermap, fill_header=None, model_variance=True)

## Write image

In [ ]:
write_image(outfile, img)

## Compare image

Differences in `OSTEP[ABCD]` can be ignored.  This changed in PR [#1543](https://github.com/desihub/desispec/pull/1543) relative to `f3`.

In [26]:
command = ['fitsdiff', '-c', 'CHECKSUM,DATASUM', outfile, os.path.join(os.path.dirname(fibermap), os.path.basename(outfile))]
p = sub.Popen(command, stdout=sub.PIPE, stderr=sub.PIPE)
out, err = p.communicate()
if out:
    print(out.decode('ascii'))
if err:
    print(out.decode('ascii'))


 fitsdiff: 4.0.1.post1
 a: /global/cscratch1/sd/bweaver/preproc-r0-00084523.fits
 b: /global/cfs/cdirs/desi/spectro/redux/f3/preproc/20210412/00084523/preproc-r0-00084523.fits
 Keyword(s) whose comments are not to be compared:
  CHECKSUM DATASUM
 Maximum number of different data values to be reported: 10
 Relative tolerance: 0.0, Absolute tolerance: 0.0

Primary HDU:

   Headers contain differences:
     Headers have different number of cards:
      a: 548
      b: 545
     Extra keyword 'COMMENT' in a: "  FITS (Flexible Image Transport System) format is defined in 'Astronomy"
     Extra keyword 'EQUINOX' in a: 2000.0
     Extra keyword 'LONGSTRN' in a: 'OGIP 1.0'
     Extra keyword 'EPOCH'  in b: 2000.0
     Keyword CHECKSUM has different values:
        a> KFdZL9ZZKEdZK9ZZ
        b> 9lMgAkJZ9kJfAkJZ
     Keyword DEPVER17 has different values:
        a> 0.48.1
        b> 0.47.1.dev6182
     Keyword OSTEPA   has different values:
        a> 0.725162289410946
        b> 0.62965874851

## Break down `read_raw`

In [16]:
def test_read_raw(filename, camera, fibermapfile=None, fill_header=None, **kwargs):
    '''Returns preprocessed raw data from `camera` extension of `filename`.
    
    Parameters
    ----------
    filename : :class:`str`
        Input FITS filename with DESI raw data.
    camera : :class:`str`
        Camera name (B0, R1, ... Z9) or FITS extension name.
    fibermapfile : :class:`str`, optional
        Read fibermap from this file; if ``None`` create blank fibermap.
    fill_header : :class:`list`, optional
        A list of HDU names or numbers.  The header cards from these HDUs
        will be added to the header of the camera HDU read from `filename`.

    Returns
    -------
    :class:`desispec.image.Image`
        Image object with member variables pix, ivar, mask, readnoise.
    
    Raises
    ------
    IOError
        If `camera` is not a HDU in `filename`.
    KeyError
        If ``EXPTIME`` is not present in any header in `filename`.
    ValueError
        If ``NIGHT`` in the primary header does not match ``NIGHT`` in the
        camera header, or if `fill_header` is not a :class:`list`.
    
    Notes
    -----
    Other keyword arguments are passed to :func:`desispec.preproc.preproc`,
    *e.g.* bias, pixflat, mask.  See :func:`~desispec.preproc.preproc`
    documentation for details.
    '''

    log = get_logger()

    t0 = time.time()
    fx = fits.open(filename, memmap=False)
    if camera.upper() not in fx:
        raise IOError(f'Camera {camera} not in {filename}!')

    rawimage = fx[camera.upper()].data
    header = fx[camera.upper()].header
    hdu = 0
    #
    # primary_header will typically represent HDU 1 ('SPEC') since
    # HDU 0 is empty.
    #
    while True:
        primary_header = fx[hdu].header
        if "EXPTIME" in primary_header: break

        if len(fx) > hdu + 1:
            if hdu > 0:
                log.warning("Did not find header keyword EXPTIME in HDU %d, moving to the next.", hdu)
            hdu += 1
        else:
            msg = "Did not find header keyword EXPTIME in any HDU of %s!"
            log.critical(msg, filename)
            raise KeyError(msg % filename)

    #- Check if NIGHT keyword is present and valid; fix if needed
    #- e.g. 20210105 have headers with NIGHT='None' instead of YEARMMDD
    try:
        tmp = int(primary_header['NIGHT'])
    except (KeyError, ValueError, TypeError):
        primary_header['NIGHT'] = (header2night(primary_header), 'Observing night')

    try:
        tmp = int(header['NIGHT'])
    except (KeyError, ValueError, TypeError):
        try:
            header['NIGHT'] = (header2night(header), 'Observing night')
        except (KeyError, ValueError, TypeError):
            #- early teststand data only have NIGHT/timestamps in primary hdr
            header['NIGHT'] = (primary_header['NIGHT'], 'Observing night')

    #- early data (e.g. 20200219/51053) had a mix of int vs. str NIGHT
    primary_header['NIGHT'] = (int(primary_header['NIGHT']), 'Observing night')
    header['NIGHT'] = (int(header['NIGHT']), 'Observing night')

    if primary_header['NIGHT'] != header['NIGHT']:
        msg = 'Primary header NIGHT=%d != camera header NIGHT=%d!'
        log.critical(msg, primary_header['NIGHT'], header['NIGHT'])
        raise ValueError(msg % (primary_header['NIGHT'], header['NIGHT']))

    #- early data have >8 char FIBERASSIGN key; rename to match current data
    if 'FIBERASSIGN' in primary_header:
        log.warning('Renaming long header keyword FIBERASSIGN -> FIBASSGN in primary_header.')
        primary_header.rename_keyword('FIBERASSIGN', 'FIBASSGN')

    if 'FIBERASSIGN' in header:
        log.warning('Renaming long header keyword FIBERASSIGN -> FIBASSGN in header.')
        header.rename_keyword('FIBERASSIGN', 'FIBASSGN')
    #
    # A lot of this inheritance stuff is moot because real data files
    # have an empty HDU 0 with no interesting headers.
    #
    inherited = False
    if 'INHERIT' in header and header['INHERIT']:
        inherited = True
        log.info('Camera header %s will INHERIT from HDU 0.', camera) 
        header.extend(fx[0].header, strip=True, unique=True)

    if fill_header is None:
        if inherited:
            hdus = []
        else:
            hdus = [0,]
        if 'PLC' in fx:
            hdus.append('PLC')
    elif isinstance(fill_header, list):
        hdus = fill_header
    else:
        msg = 'Unknown type for fill_header!'
        log.critical(msg)
        raise ValueError(msg)

    log.info('Will add header keywords from HDUs %s.', str(hdus))
    
    for hdu in hdus:
        if hdu in fx or int(hdu) in fx:
            header.extend(fx[hdu].header, strip=True, unique=True)
        else:
            log.warning("HDU %s is not in FITS file.", str(hdu))

    fx.close()
    duration = time.time() - t0
    log.info(iotime.format('read', filename, duration))
    #
    # Other cleanup of headers
    #
    longstrn = fits.Card('LONGSTRN', 'OGIP 1.0', 'The OGIP Long String Convention may be used.')
    if 'MODULE' in primary_header:
        log.debug("Inserting LONGSTRN keyword before MODULE.")
        primary_header.insert('MODULE', longstrn)
    else:
        log.debug("Inserting LONGSTRN keyword before EXTNAME.")
        primary_header.insert('EXTNAME', longstrn)
    log.debug("Renaming EPOCH to EQUINOX in primary_header.")
    primary_header.rename_keyword('EPOCH', 'EQUINOX')
    log.debug("Renaming EPOCH to EQUINOX in header.")
    header.rename_keyword('EPOCH', 'EQUINOX')
    #
    # Run preproc()
    #
    img = preproc(rawimage, header, primary_header, **kwargs)
    #
    # Load fibermap data.
    #
    if fibermapfile is not None and os.path.exists(fibermapfile):
        fibermap = read_fibermap(fibermapfile)
    else:
        log.warning('creating blank fibermap')
        fibermap = empty_fibermap(5000)

    #- Add image header keywords inherited from raw data to fibermap too
    # but I think this was already done in assemble_fibermap
    addkeys(fibermap.meta, img.meta)

    #- Augment the image header with some tile info from fibermap if needed
    for key in ['TILEID', 'TILERA', 'TILEDEC']:
        if key in fibermap.meta:
            if key not in img.meta:
                log.info('Updating header from fibermap {}={}'.format(
                    key, fibermap.meta[key]))
                img.meta[key] = fibermap.meta[key]
            elif img.meta[key] != fibermap.meta[key]:
                #- complain loudly, but don't crash and don't override
                log.error('Inconsistent {}: raw header {} != fibermap header {}'.format(key, img.meta[key], fibermap.meta[key]))


    #- Trim to matching camera based upon PETAL_LOC, but that requires
    #- a mapping prior to 20191211

    #- HACK HACK HACK
    #- TODO: replace this with a mapping from calibfinder, as soon as
    #- that is implemented in calibfinder / desi_spectro_calib
    #- HACK HACK HACK

    #- From DESI-5286v5 page 3 where sp=sm-1 and
    #- "spectro logical number" = petal_loc
    spec_to_petal = {4:2, 2:9, 3:0, 5:3, 1:8, 0:4, 6:6, 7:7, 8:5, 9:1}
    assert set(spec_to_petal.keys()) == set(range(10))
    assert set(spec_to_petal.values()) == set(range(10))

    #- Mapping only for dates < 20191211
    if "NIGHT" in primary_header:
        dateobs = int(primary_header["NIGHT"])
    elif "DATE-OBS" in primary_header:
        dateobs=parse_date_obs(primary_header["DATE-OBS"])
    else:
        msg = "Need either NIGHT or DATE-OBS in primary header"
        log.error(msg)
        raise KeyError(msg)
    if dateobs < 20191211 :
        petal_loc = spec_to_petal[int(camera[1])]
        log.warning('Prior to 20191211, mapping camera {} to PETAL_LOC={}'.format(camera, petal_loc))
    else :
        petal_loc = int(camera[1])
        log.debug('Since 20191211, camera {} is PETAL_LOC={}'.format(camera, petal_loc))

    if 'PETAL_LOC' in fibermap.dtype.names : # not the case in early teststand data
        ii = (fibermap['PETAL_LOC'] == petal_loc)
        fibermap = fibermap[ii]

    cfinder = None

    camname = camera.upper()[0]
    if camname == 'B':
        badamp_bit = maskbits.fibermask.BADAMPB
    elif camname == 'R':
        badamp_bit = maskbits.fibermask.BADAMPR
    else:
        badamp_bit = maskbits.fibermask.BADAMPZ


    if 'FIBER' in fibermap.dtype.names : # not the case in early teststand data

        ## Mask fibers
        cfinder = CalibFinder([header,primary_header])
        fibers  = fibermap['FIBER'].data
        for k in ["BROKENFIBERS","BADCOLUMNFIBERS","LOWTRANSMISSIONFIBERS"] :
            log.debug("{}={}".format(k,cfinder.badfibers([k])))

        ## Mask bad fibers
        fibermap['FIBERSTATUS'][np.in1d(fibers,cfinder.badfibers(["BROKENFIBERS"]))] |= maskbits.fibermask.BROKENFIBER
        fibermap['FIBERSTATUS'][np.in1d(fibers,cfinder.badfibers(["BADCOLUMNFIBERS"]))] |= maskbits.fibermask.BADCOLUMN
        fibermap['FIBERSTATUS'][np.in1d(fibers,cfinder.badfibers(["LOWTRANSMISSIONFIBERS"]))] |= maskbits.fibermask.LOWTRANSMISSION
        # Also, for backward compatibility
        fibermap['FIBERSTATUS'][np.in1d(fibers%500,cfinder.badfibers(["BROKENFIBERS"])%500)] |= maskbits.fibermask.BROKENFIBER
        fibermap['FIBERSTATUS'][np.in1d(fibers%500,cfinder.badfibers(["BADCOLUMNFIBERS"])%500)] |= maskbits.fibermask.BADCOLUMN
        fibermap['FIBERSTATUS'][np.in1d(fibers%500,cfinder.badfibers(["LOWTRANSMISSIONFIBERS"])%500)] |= maskbits.fibermask.LOWTRANSMISSION

        # Mask Fibers that are set to be excluded due to CCD/amp/readout issues
        fibermap['FIBERSTATUS'][np.in1d(fibers,cfinder.badfibers(["BADAMPFIBERS"]))] |= badamp_bit
        fibermap['FIBERSTATUS'][np.in1d(fibers,cfinder.badfibers(["EXCLUDEFIBERS"]))] |= badamp_bit # for backward compatibiliyu
        fibermap['FIBERSTATUS'][np.in1d(fibers%500,cfinder.badfibers(["BADAMPFIBERS"])%500)] |= badamp_bit
        fibermap['FIBERSTATUS'][np.in1d(fibers%500,cfinder.badfibers(["EXCLUDEFIBERS"])%500)] |= badamp_bit # for backward compatibiliyu
        if cfinder.haskey("EXCLUDEFIBERS") :
            log.warning("please use BADAMPFIBERS instead of EXCLUDEFIBERS")

    if np.sum(img.mask & maskbits.ccdmask.BADREADNOISE > 0) >= img.mask.size//4 :
        log.info("Propagate ccdmask.BADREADNOISE to fibermap FIBERSTATUS")

        if cfinder is None :
            cfinder = CalibFinder([header,primary_header])

        psf_filename = cfinder.findfile("PSF")
        tset = read_xytraceset(psf_filename)
        mean_wave =(tset.wavemin+tset.wavemax)/2.
        xfiber  = tset.x_vs_wave(np.arange(tset.nspec),mean_wave)
        amp_ids = desispec.preproc.get_amp_ids(header)

        for amp in amp_ids :
            kk  = parse_sec_keyword(header['CCDSEC'+amp])
            ntot = img.mask[kk].size
            nbad = np.sum((img.mask[kk] & maskbits.ccdmask.BADREADNOISE) > 0)
            if nbad / ntot > 0.5 :
                # not just nbad>0 b/c/ there are always pixels with low QE
                # that have increased readnoise after pixel flatfield
                log.info("Setting BADREADNOISE bit for fibers of amp {}".format(amp))
                badfibers = (xfiber>=kk[1].start-3)&(xfiber<kk[1].stop+3)
                fibermap["FIBERSTATUS"][badfibers] |= ( maskbits.fibermask.BADREADNOISE | badamp_bit )

    img.fibermap = fibermap

    return img


In [17]:
img = test_read_raw(infile, camera, fibermapfile=fibermap, fill_header=None, model_variance=True)

INFO:<ipython-input-16-e6c4137c090e>:122:test_read_raw: Will add header keywords from HDUs [0].
INFO:<ipython-input-16-e6c4137c090e>:132:test_read_raw: iotime 0.496 sec to read desi-00084523.fits.fz at 2022-01-05T15:41:09.174763
DEBUG:<ipython-input-16-e6c4137c090e>:138:test_read_raw: Inserting LONGSTRN keyword before MODULE.
DEBUG:<ipython-input-16-e6c4137c090e>:143:test_read_raw: Renaming EPOCH to EQUINOX in primary_header.
DEBUG:<ipython-input-16-e6c4137c090e>:145:test_read_raw: Renaming EPOCH to EQUINOX in header.
DEBUG:calibfinder.py:182:__init__: header['CAMERA']=r0
DEBUG:calibfinder.py:186:__init__: header['SPECID']=4
DEBUG:calibfinder.py:236:__init__: Use spectrograph hardware identifier SMY
DEBUG:calibfinder.py:246:__init__: reading calib data in /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm4/sm4-r.yaml
DEBUG:calibfinder.py:258:__init__: Found 6 data for camera sm4-r in filename /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm4/sm4-r.yaml
DEBUG

## Break down `write_image`

In [24]:
def test_write_image(outfile, image, meta=None):
    """Writes image object to outfile

    Args:
        outfile : output file string
        image : desispec.image.Image object
            (or any object with 2D array attributes image, ivar, mask)

    Optional:
        meta : dict-like object with metadata key/values (e.g. FITS header)
    """

    log = get_logger()
    if meta is not None:
        hdr = fitsheader(meta)
    else:
        hdr = fitsheader(image.meta)

    add_dependencies(hdr)

    #- Work around fitsio>1.0 writing blank keywords, e.g. on 20191212
    for key in hdr.keys():
        if type(hdr[key]) == fits.card.Undefined:
            log.warning('Setting blank keyword {} to None'.format(key))
            hdr[key] = None

    outdir = os.path.dirname(os.path.abspath(outfile))
    if not os.path.isdir(outdir):
        os.makedirs(outdir, exist_ok=True)

    hx = fits.HDUList()
    hdu = fits.ImageHDU(image.pix.astype(np.float32), name='IMAGE', header=hdr)
    if 'CAMERA' not in hdu.header:
        hdu.header.append( ('CAMERA', image.camera.lower(), 'Spectrograph Camera') )

    if 'RDNOISE' not in hdu.header and np.isscalar(image.readnoise):
        hdu.header.append( ('RDNOISE', image.readnoise, 'Read noise [RMS electrons/pixel]'))

    hx.append(hdu)
    hx.append(fits.ImageHDU(image.ivar.astype(np.float32), name='IVAR'))
    hx.append(fits.CompImageHDU(image.mask.astype(np.int16), name='MASK'))
    if not np.isscalar(image.readnoise):
        hx.append(fits.ImageHDU(image.readnoise.astype(np.float32), name='READNOISE'))

    if hasattr(image, 'fibermap'):
        if isinstance(image.fibermap, Table):
            with warnings.catch_warnings():
                #- nanomaggies aren't an official IAU unit but don't complain
                warnings.filterwarnings('ignore', ".*nanomaggies.*")
                fmhdu = fits.convenience.table_to_hdu(image.fibermap)
            fmhdu.name = 'FIBERMAP'
        else:
            fmhdu = fits.BinTableHDU(image.fibermap, name='FIBERMAP')

        hx.append(fmhdu)

    t0 = time.time()
    hx.writeto(outfile+'.tmp', overwrite=True, checksum=True)
    os.rename(outfile+'.tmp', outfile)
    duration = time.time() - t0
    log.info(iotime.format('write', outfile, duration))

    return outfile


In [25]:
test_write_image(outfile, img)

INFO:<ipython-input-24-6e9d0c1c1e03>:61:test_write_image: iotime 63.597 sec to write preproc-r0-00084523.fits at 2022-01-05T15:45:46.862792


'/global/cscratch1/sd/bweaver/preproc-r0-00084523.fits'

## Test fitsio on fibermap files

In [ ]:
# Huh, fitsio changes the order of comment cards.

In [4]:
fmap, fmap_hdr = fitsio.read(fibermap, ext='FIBERMAP', header=True)

In [5]:
fmap_hdr


XTENSION= 'BINTABLE'           / binary table extension
BITPIX  =                    8 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                  385 / length of dimension 1
NAXIS2  =                 5000 / length of dimension 2
PCOUNT  =                    0 / number of group parameters
GCOUNT  =                    1 / number of groups
TFIELDS =                   73 / number of table fields
TTYPE1  = 'TARGETID'           / 
TFORM1  = 'K'                  / 
TTYPE2  = 'PETAL_LOC'          / 
TFORM2  = 'I'                  / 
TTYPE3  = 'DEVICE_LOC'         / 
TFORM3  = 'J'                  / 
TTYPE4  = 'LOCATION'           / 
TFORM4  = 'K'                  / 
TTYPE5  = 'FIBER'              / 
TFORM5  = 'J'                  / 
TTYPE6  = 'FIBERSTATUS'        / 
TFORM6  = 'J'                  / 
TTYPE7  = 'TARGET_RA'          / 
TFORM7  = 'D'                  / 
TTYPE8  = 'TARGET_DEC'         / 
TFORM8  = 'D'                  / 
TTYPE9  = 'PMR

In [10]:
fits.Header.fromstring(str(fmap_hdr).lstrip(), sep='\n')

 [astropy.io.fits.verify]


XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  385 / length of dimension 1                          
NAXIS2  =                 5000 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   73 / number of table fields                         
TTYPE1  = 'TARGETID'           /                                                
TFORM1  = 'K'                  /                                                
TTYPE2  = 'PETAL_LOC'          /                                                
TFORM2  = 'I'                  /                                                
TTYPE3  = 'DEVICE_LOC'      

In [7]:
fmap_table = Table(fmap)

In [8]:
fmap_table

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,PLATE_RA,PLATE_DEC,NUM_ITER,FIBER_X,FIBER_Y,DELTA_X,DELTA_Y,FIBER_RA,FIBER_DEC,EXPTIME
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,str3,float32,float32,int32,float64,int32,int16,str8,int32,int32,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64
39627818581100303,0,311,311,0,0,217.81547483555607,1.3157446561897221,0.0,0.0,2015.5,5400.0,65537,1,TGT,86.092896,-287.40054,103200,0.9734380521456845,1,9010,2178p012,338439,1807,DEV,0.039295673,0.2611657,1.6636531,8.58879,28.541748,13.060156,393.6064,132.85022,31.536251,2.3298013,0.57979095,0.09827473,0.62602043,3.2318983,0.11390534,0.6510575,3.2725024,0,4.0,0.89585626,0.022532921,-0.104067504,0,,0.0,0.0,0.0,0.0,S,103200,9,65537,0,0,0,0,0,0,217.81547483555607,1.3157446561897221,2,86.067,-287.339,-0.004,-0.003,217.81545868019776,1.3157564374777024,677.6638
39627818581103159,0,272,272,1,0,217.93845192938744,1.3583965003591383,0.0,0.0,2015.5,5400.0,1179714,1,TGT,55.34799,-275.97507,103100,0.8875012356025477,1,9010,2178p012,338439,4663,PSF,0.043902166,0.31993815,0.335153,0.6139619,-0.76292324,-0.60965633,1264.1632,674.43506,159.8648,3.5092566,0.7400335,0.24922459,0.2610766,0.47826242,0.24922459,0.2610766,0.47826242,0,0.0,0.0,0.0,0.0,0,,0.0,0.0,0.0,0.0,S,103100,9,1179714,0,0,0,0,0,0,217.93845192938744,1.3583965003591383,2,55.324,-275.911,-0.001,-0.005,217.9384477755994,1.3584159298852077,677.6638
39627824625091229,0,252,252,2,0,218.0307597285977,1.3836031112254092,0.0,0.0,2015.5,5400.0,1179714,1,TGT,32.401966,-269.29184,103100,0.15539519201178198,1,9010,2181p015,339880,669,REX,0.04513969,0.35698873,0.4086271,0.76048607,1.6238836,1.1266618,753.32935,293.94193,104.49555,3.464112,0.7072578,0.2515084,0.28788906,0.53578347,0.2515084,0.28788906,0.53578347,0,1.0,0.21876171,0.0,0.0,0,,0.0,0.0,0.0,0.0,S,103100,9,1179714,0,0,0,0,0,0,218.0307597285977,1.3836031112254092,2,32.384,-269.227,-0.004,-0.007,218.03074350295427,1.3836304426315482,677.6638
39627824625094434,0,156,156,3,0,218.15523369529456,1.591644978113127,0.0,0.0,2015.5,5400.0,655394,1,TGT,1.61792,-216.14743,103000,0.5452548327351018,1,9010,2181p015,339880,3874,REX,0.04144086,0.66480297,0.91725636,2.212188,2.3458712,2.4881804,941.2605,448.03506,90.40519,3.487175,0.7145109,0.38939536,0.537265,1.2957458,0.38939536,0.537265,1.2957458,0,1.0,0.3921881,0.0,0.0,0,,0.0,0.0,0.0,0.0,S,103000,9,655394,0,0,0,0,0,0,218.15523369529456,1.591644978113127,2,1.596,-216.091,0.005,-0.012,218.15525407210612,1.5916925926170828,677.6638
39627824629285168,0,198,198,4,0,218.26037016321155,1.4486134973950568,-6.1616716,-5.1620083,2015.5,5400.0,2305843017803628544,3,TGT,-24.408787,-252.49036,0,0.640657447472838,519,9010,2183p015,339881,304,PSF,0.042178728,29.072124,39.645054,44.217606,10.597573,1.2729639,299.1595,200.65352,96.93992,3.3007205,0.7394041,22.563866,30.769876,34.31879,22.563866,30.769876,34.31879,0,0.0,0.0,0.0,0.0,3655808130073825920,G2,18.573793,18.871632,18.109484,0.07758351,S,-1,-1,2305843017803628544,0,1280,0,0,0,0,218.

In [9]:
fmap_table.meta

OrderedDict()

In [11]:
fibermap

'/global/cfs/cdirs/desi/spectro/redux/f3/preproc/20210412/00084523/fibermap-00084523.fits'

In [24]:
img = fits.PrimaryHDU(np.random.uniform(size=(10,10)))
img.header['COMMENT'] = 'This is the first comment.'
img.header['COMMENT'] = 'This is the second comment.'
img.header['HISTORY'] = 'This is the first history.'
img.header['HISTORY'] = 'This is the second history.'
img.header['EXTNAME'] = 'TEST'
img.header['PLANCK'] = (6.62607015e-34, "[J s] Planck's Constant")
hdulist = fits.HDUList([img])
hdulist.writeto(os.path.join(os.environ['CSCRATCH'], 'fitsio_header_test.fits'), overwrite=True)

In [28]:
img.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   10                                                  
NAXIS2  =                   10                                                  
EXTEND  =                    T                                                  
EXTNAME = 'TEST    '                                                            
PLANCK  =       6.62607015E-34 / [J s] Planck's Constant                        
COMMENT This is the first comment.                                              
COMMENT This is the second comment.                                             
HISTORY This is the first history.                                              
HISTORY This is the second history.                                             

In [25]:
t, hdr = fitsio.read(os.path.join(os.environ['CSCRATCH'], 'fitsio_header_test.fits'), ext='TEST', header=True)

In [26]:
t

array([[0.93629373, 0.50402912, 0.33338373, 0.78207632, 0.91477867,
        0.61516523, 0.20176118, 0.74318738, 0.42151287, 0.52842448],
       [0.65350741, 0.34360757, 0.18054487, 0.4823943 , 0.95265483,
        0.83465029, 0.07968847, 0.02792331, 0.55392739, 0.95344132],
       [0.68611458, 0.93750244, 0.10821171, 0.71305134, 0.13256958,
        0.01472093, 0.81384076, 0.46308861, 0.0448992 , 0.54122625],
       [0.16188969, 0.58412355, 0.28261036, 0.18016786, 0.34175302,
        0.81203808, 0.20162513, 0.22807459, 0.9497739 , 0.29727654],
       [0.02695568, 0.0360019 , 0.8326586 , 0.56051252, 0.15325676,
        0.13841857, 0.29202632, 0.03902353, 0.83296968, 0.16560177],
       [0.12848082, 0.33845486, 0.23295145, 0.99953017, 0.35004202,
        0.13728355, 0.22354982, 0.77492993, 0.85051572, 0.00120422],
       [0.99692731, 0.56367267, 0.79068865, 0.25895647, 0.87001734,
        0.84655542, 0.60865723, 0.89820939, 0.62951146, 0.57418039],
       [0.11714111, 0.10463774, 0.7929024

In [34]:
hdr


SIMPLE  =                    T / conforms to FITS standard
BITPIX  =                  -64 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                   10 / 
NAXIS2  =                   10 / 
EXTEND  =                    T / 
EXTNAME = 'TEST'               / 
PLANCK  =       6.62607015e-34 / [J s] Planck's Constant
COMMENT This is the second comment.
COMMENT This is the first comment.
HISTORY   This is the second history.
HISTORY   This is the first history.

In [35]:
repr(hdr)

"\nSIMPLE  =                    T / conforms to FITS standard\nBITPIX  =                  -64 / array data type\nNAXIS   =                    2 / number of array dimensions\nNAXIS1  =                   10 / \nNAXIS2  =                   10 / \nEXTEND  =                    T / \nEXTNAME = 'TEST'               / \nPLANCK  =       6.62607015e-34 / [J s] Planck's Constant\nCOMMENT This is the second comment.\nCOMMENT This is the first comment.\nHISTORY   This is the second history.\nHISTORY   This is the first history."